## Run N4BiasFieldCorrectionImageFilter
on Train and validation of Brats data


## Data files
* /naomi/DATA/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA04_192_1
```
BraTS19_TCIA04_192_1_flair.nii.gz
BraTS19_TCIA04_192_1_t1.nii.gz    
BraTS19_TCIA04_192_1_t2.nii.gz
BraTS19_TCIA04_192_1_seg.nii.gz    
BraTS19_TCIA04_192_1_t1ce.nii.gz
```
After running the filter the following files will be added:
```
BraTS19_TCIA09_141_1_t1_nb4.nii.gz
BraTS19_TCIA09_141_1_t1ce.nii.gz
```

In [2]:
import paths

path_names = paths.set_names()

DATA_HGG = path_names[2]
DATA_LGG = path_names[3]
VALIDATION_DATA = path_names[0]

FLAIR = 'flair'
T1 = 't1'
T2 = 't2'
T1CE = 't1ce'

reduced_Data_HGG = path_names[4]
reduced_Data_LGG = path_names[5]
reduced_Val = path_names[1]
reduced_VALIDATION_NUMPY_DIR = path_names[7]



In [3]:
import os, sys, glob
import numpy as np
import SimpleITK as sitk
import sys
import os
import pandas as pd
import re

In [4]:
import pandas
import numpy

## Data Preprocessing
### Run  N4BiasFieldCorrectionImageFilter

In [5]:
# reduce bias 
def n4cor(in_img, out_img):
    inputImage = sitk.ReadImage(in_img )
    maskImage = sitk.OtsuThreshold( inputImage, 0, 1, 200 )

    inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter();
    numberFilltingLevels = 4

    output = corrector.Execute( inputImage, maskImage ) 
    sitk.WriteImage( output, out_img )

#### find all relevant image directories

In [6]:
# Deletes all files in a folder
# https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
# import shutil, random, os
# os.mkdir(Shrinked_Data_LGG)
#
# for filename in os.listdir(Shrinked_Data_LGG):
#     file_path = os.path.join(Shrinked_Data_LGG, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))


In [7]:
# Our laptops lack computing power to make full use of the data set.
# we reduce the data set to 1/10 of its original size so that we can play with and learn
# from the different data sets we realize that this effects accuracy of our findings, 
# but otherwise many of the different functions takes hours to run and even with reduced
# data it still takes hours to run some functions do to imcompatabe cuda GPUs and slow CPUs

# reduce LGG data
import random
input_path = DATA_HGG
hgg_paths = []
for dirpath, dirnames, files in os.walk(input_path):
    for dir in dirnames:
        if ('BraTS19' in dir):
            hgg_paths.append(dir)

# random_hgg = np.random.choice(hgg_paths, int(len(hgg_paths)*.1))
# hgg_paths = random_hgg

input_path_LGG = DATA_LGG
lgg_paths = []
for dirpath, dirnames, files in os.walk(input_path):
    for dir in dirnames:
        if ('BraTS19' in dir):
            lgg_paths.append(dir)

random_lgg = np.random.choice(lgg_paths, int(len(lgg_paths)*.1))
lgg_paths = random_lgg
import shutil, random, os

#reduce LGG
#https://stackoverflow.com/questions/15034151/copy-directory-contents-into-a-directory-with-python
import shutil, random, os
import os, shutil
from distutils.dir_util import copy_tree
dirLGG = DATA_LGG
LGG_Names = random.sample(os.listdir(dirLGG), 7)
for fname in LGG_Names:
    srcpath = os.path.join(dirLGG, fname)
    path = os.path.join(reduced_Data_LGG, fname)
    os.mkdir(path)
    copy_tree(srcpath, path)
    


FileExistsError: [Errno 17] File exists: '/Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/MICCAI_BraTS_2019_Data_Training/reducedLGG/BraTS19_TCIA10_307_1'

In [8]:
## reduce HGG data
dirHGG = DATA_HGG
HGG_Names = random.sample(os.listdir(dirHGG), 25)
for fname in HGG_Names:
    srcpath = os.path.join(dirHGG, fname)
    path = os.path.join(reduced_Data_HGG, fname)
    os.mkdir(path)
    copy_tree(srcpath, path)

FileExistsError: [Errno 17] File exists: '/Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/MICCAI_BraTS_2019_Data_Training/reducedHGG/BraTS19_CBICA_AWI_1'

In [9]:
# reduce validation data
Val_Names = random.sample(os.listdir(VALIDATION_DATA), 12)
for fname in Val_Names:
    srcpath = os.path.join(VALIDATION_DATA, fname)
    path = os.path.join(reduced_Val, fname)
    os.mkdir(path)
    copy_tree(srcpath, path)

### Run n4corr filter on T1 and T1CE images
Time on Train data
```1
CPU times: user 2d 11h 24min 48s, sys: 47min 15s, total: 2d 12h 12min 3s
Wall time: 16h 43min 29s
```

In [10]:
%%time
# run n4 on training
for dirpath, dirnames, files in os.walk(reduced_Data_LGG):

    for file in files:
        if ('_t1.' in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath, file[:-9]+ 't1_nb4.nii.gz')
            print(in_img, out_img)
            if (not os.path.isfile(out_img)):
                n4cor(in_img, out_img)
        
        if (T1CE in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath,file[:-11]+ 't1ce_nb4.nii.gz')
            print(in_img, out_img)
            if (not os.path.isfile(out_img)):
                n4cor(in_img, out_img)

for dirpath, dirnames, files in os.walk(reduced_Data_HGG):

    for file in files:
        if ('_t1.' in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath, file[:-9]+ 't1_nb4.nii.gz')
            print(in_img, out_img)
            if (not os.path.isfile(out_img)):
                n4cor(in_img, out_img)

        if (T1CE in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath,file[:-11]+ 't1ce_nb4.nii.gz')
            print(in_img, out_img)
            if (not os.path.isfile(out_img)):
                n4cor(in_img, out_img)

/Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/MICCAI_BraTS_2019_Data_Training/reducedLGG/BraTS19_TCIA10_202_1/BraTS19_TCIA10_202_1_t1.nii.gz /Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/MICCAI_BraTS_2019_Data_Training/reducedLGG/BraTS19_TCIA10_202_1/BraTS19_TCIA10_202_1_t1_nb4.nii.gz
/Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/MICCAI_BraTS_2019_Data_Training/reducedLGG/BraTS19_TCIA10_202_1/BraTS19_TCIA10_202_1_t1ce_nb4.nii.gz /Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/MICCAI_BraTS_2019_Data_Training/reducedLGG/BraTS19_TCIA10_202_1/BraTS19_TCIA10_202_1_t1cet1ce_nb4.nii.gz


KeyboardInterrupt: 

In [11]:
%%time
# run n4 on validation data
for dirpath, dirnames, files in os.walk(reduced_Val):

    for file in files:
        if ('_t1.' in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath, file[:-9]+ 't1_nb4.nii.gz')
            if (not os.path.isfile(out_img)):
                print(in_img, out_img)
                n4cor(in_img, out_img)
        
        if (T1CE in file):
            in_img = os.path.join(dirpath, file)
            out_img = os.path.join(dirpath,file[:-11]+ 't1ce_nb4.nii.gz')
            if (not os.path.isfile(out_img)):
                print(in_img, out_img)
                n4cor(in_img, out_img)

/Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/reducedVal/testReduce/BraTS19_UAB_3455_1/BraTS19_UAB_3455_1_t1.nii.gz /Users/nathansimms/Documents/CMPT340/BratsData/2019Simple/reducedVal/testReduce/BraTS19_UAB_3455_1/BraTS19_UAB_3455_1_t1_nb4.nii.gz


KeyboardInterrupt: 